# 2. Extracción Automatizada de Documentos PDF

Este cuaderno está dedicado a la descarga automatizada de documentos normativos en formato PDF, a partir de un archivo consolidado de metadatos.

Los principales objetivos de esta etapa son:

- Cargar y filtrar el conjunto de datos según parámetros definidos como tipo de documento y año.
- Validar la presencia y el formato correcto de las URLs asociadas a cada registro.
- Descargar los archivos PDF correspondientes solo si no han sido previamente almacenados.
- Guardar los archivos con nombres estandarizados a partir del campo `file_name`.

El script asegura la trazabilidad del proceso manteniendo una estructura de carpetas limpia y registrando mensajes de error cuando una descarga falla o el recurso no está disponible.

> Nota: El archivo de metadatos debe contener las columnas `document_type`, `year`, `file_name` y `url`.


In [18]:
#library
import os
import pandas as pd
import requests
from tqdm import tqdm
import re

In [19]:
# -------------------------------
# CONSTANTS
# -------------------------------
DATA_DIR = "./../raw_data"
PDF_DIR = os.path.join(DATA_DIR, "pdfs")
SUFFIX = ""
FILE_NAME = f"combined_regulations.csv"
METADATA_FILE = os.path.join(DATA_DIR, FILE_NAME)
TARGET_TYPES = ['circular', 'resolution']

In [ ]:
os.makedirs(PDF_DIR, exist_ok=True)

FILE_TEMPLATES = {
    'circular': f'circulares_{SUFFIX}.csv',
    'resolution': f'resoluciones_{SUFFIX}.csv'
}
df_meta = pd.read_csv(METADATA_FILE)


# -------------------------------
# Download PDFs
# -------------------------------
print(f"Total files to download: {len(df_meta)}")

for _, row in tqdm(df_meta.iterrows(), total=len(df_meta), desc="Downloading PDFs"):
    url = row['url']
    if pd.isna(url) or not isinstance(url, str) or not url.startswith("http"):
        continue

    file_name = row['nombre_archivo'] + ".pdf"
    local_path = os.path.join(PDF_DIR, file_name)

    if not os.path.exists(local_path):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                with open(local_path, 'wb') as f:
                    f.write(response.content)
            else:
                print(f"⚠️ Error {response.status_code} while downloading {url}")
        except Exception as e:
            print(f"❌ Exception with {url}: {e}")
    else:
        print(f"✓ Already downloaded: {file_name}")
        


## 2.1 Conclusiones

En esta segunda etapa del proceso de análisis documental, se desarrolló una estrategia automatizada para la descarga masiva de documentos normativos en formato PDF, utilizando como referencia un archivo consolidado de metadatos previamente generado. Este procedimiento fue clave para garantizar la trazabilidad, integridad y sistematización de la base documental de estudio.

El flujo de trabajo consistió en cargar y filtrar el conjunto de metadatos, validando la estructura y presencia de URLs asociadas a cada registro. Se implementó un mecanismo  que permitió descargar únicamente aquellos archivos no existentes previamente en el repositorio local, estandarizando sus nombres a partir del identificador de metadatos para mantener la organización y facilitar la posterior búsqueda o análisis.

El sistema registró los errores o incidencias durante la descarga, permitiendo así una revisión y control de calidad eficiente sobre el conjunto de archivos obtenidos. La automatización del proceso no solo optimizó el tiempo de recopilación y redujo el margen de error humano, sino que sentó las bases para una gestión documental reproducible y escalable en futuras investigaciones.
